In [8]:
#CONFIG INICIAL

import requests
import pandas as pd
import os
import time

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
HEADERS = {"accept": "application/json"}
RAW_DIR = "dados/raw"
os.makedirs(RAW_DIR, exist_ok=True)

def get_json(url, params=None):
    try:
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"[ERRO] Status {response.status_code} - {url}")
    except Exception as e:
        print(f"[EXCEÇÃO] {e}")
    return None

In [2]:
# 1. Deputados
def get_deputados():
    print("[INICIANDO] Coleta de deputados...")
    url = f"{BASE_URL}/deputados"
    data = get_json(url, {"itens": 1000})
    
    if data and "dados" in data:
        df = pd.DataFrame(data["dados"])
        path = os.path.join(RAW_DIR, "deputados.csv")
        df.to_csv(path, index=False)
        print(f"[FINALIZADO] {len(df)} deputados salvos em '{path}'")
        return df
    else:
        print("[ERRO] Nenhum dado retornado.")
        return pd.DataFrame()

In [3]:
df_deputados = get_deputados()
df_deputados.head()

[INICIANDO] Coleta de deputados...
[FINALIZADO] 513 deputados salvos em 'dados/raw/deputados.csv'


,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
1,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
2,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
3,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br
4,204528,https://dadosabertos.camara.leg.br/api/v2/depu...,Adriana Ventura,NOVO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adrianaventura@camara.leg.br


In [4]:
# 2. Partidos
def get_partidos():
    print("[INICIANDO] Coleta de partidos...")
    url = f"{BASE_URL}/partidos"
    data = get_json(url)
    
    if data and "dados" in data:
        df = pd.DataFrame(data["dados"])
        path = os.path.join(RAW_DIR, "partidos.csv")
        df.to_csv(path, index=False)
        print(f"[FINALIZADO] {len(df)} partidos salvos em '{path}'")
        return df
    else:
        print("[ERRO] Nenhum partido encontrado.")
        return pd.DataFrame()

In [5]:
df_partidos = get_partidos()
df_partidos.head()

[INICIANDO] Coleta de partidos...
[FINALIZADO] 15 partidos salvos em 'dados/raw/partidos.csv'


,id,sigla,nome,uri
0,36898,AVANTE,Avante,https://dadosabertos.camara.leg.br/api/v2/part...
1,37905,CIDADANIA,Cidadania,https://dadosabertos.camara.leg.br/api/v2/part...
2,36899,MDB,Movimento Democrático Brasileiro,https://dadosabertos.camara.leg.br/api/v2/part...
3,37901,NOVO,Partido Novo,https://dadosabertos.camara.leg.br/api/v2/part...
4,36779,PCdoB,Partido Comunista do Brasil,https://dadosabertos.camara.leg.br/api/v2/part...


In [9]:
# 3. Despesas por deputado
def get_despesas_deputados(ano=2023):
    print(f"[INICIANDO] Coleta de despesas dos deputados para o ano {ano}...")

    path_deputados = os.path.join(RAW_DIR, "deputados.csv")
    if not os.path.exists(path_deputados):
        print("[ERRO] Arquivo de deputados não encontrado. Rode primeiro a coleta de deputados.")
        return pd.DataFrame()
    
    deputados_df = pd.read_csv(path_deputados)
    despesas_total = []

    for i, row in deputados_df.iterrows():
        dep_id = row['id']
        dep_nome = row['nome']
        pagina = 1

        while True:
            url = f"{BASE_URL}/deputados/{dep_id}/despesas"
            params = {"ano": ano, "pagina": pagina, "itens": 100}
            data = get_json(url, params)

            if not data or "dados" not in data or not data["dados"]:
                break

            for item in data["dados"]:
                despesas_total.append({
                    "id_deputado": dep_id,
                    "nome_deputado": dep_nome,
                    "tipo_despesa": item.get("tipoDespesa"),
                    "fornecedor": item.get("nomeFornecedor"),
                    "valor": item.get("valorDocumento"),
                    "data": item.get("dataDocumento"),
                    "documento": item.get("numDocumento")
                })

            pagina += 1
            time.sleep(0.02)  # Para evitar sobrecarregar a API

        if (i+1) % 50 == 0:
            print(f"[INFO] Deputado {i+1}/{len(deputados_df)} processado.")

    df_despesas = pd.DataFrame(despesas_total)
    path_out = os.path.join(RAW_DIR, f"despesas_{ano}.csv")
    df_despesas.to_csv(path_out, index=False)
    print(f"[FINALIZADO] {len(df_despesas)} despesas salvas em '{path_out}'")

    return df_despesas

In [10]:
df_despesas = get_despesas_deputados(2023)
df_despesas.head()

[INICIANDO] Coleta de despesas dos deputados para o ano 2023...
[INFO] Deputado 50/513 processado.


KeyboardInterrupt: 